In [78]:
import pandas as pd

df=pd.read_csv('merged_training_table.csv')
df.head()

,month,sector,year,amount_new_house_transactions,year_end_registered_population_10k,total_households_10k,year_end_resident_population_10k,year_end_total_employed_population_10k,year_end_urban_non_private_employees_10k,private_individual_and_other_employees_10k,...,medical_health_rehabilitation_institution_dense,medical_health_first_aid_center_dense,medical_health_blood_donation_station_dense,medical_health_disease_prevention_institution_dense,medical_health_general_hospital_dense,medical_health_clinic_dense,education_training_school_education_middle_school_dense,education_training_school_education_primary_school_dense,education_training_school_education_kindergarten_dense,education_training_school_education_research_institution_dense
0,2019-01-01,sector 1,2019,13827.14,953.72,313.8455,1530.59,1125.89,400.22,725.672,...,0.000113,0.0,0.0,8.600000e-07,0.000041,0.000038,0.000016,0.000028,0.000063,0.000014
1,2019-01-01,sector 1,2019,13827.14,953.72,313.8455,1530.59,1125.89,400.22,725.672,...,0.000113,0.0,0.0,8.600000e-07,0.000041,0.000038,0.000016,0.000028,0.000063,0.000014
2,2019-01-01,sector 1,2019,13827.14,953.72,313.8455,1530.59,1125.89,400.22,725.672,...,0.000113,0.0,0.0,8.600000e-07,0.000041,0.000038,0.000016,0.000028,0.000063,0.000014
3,2019-01-01,sector 1,2019,13827.14,953.72,313.8455,1530.59,1125.89,400.22,725.672,...,0.000113,0.0,0.0,8.600000e-07,0.000041,0.000038,0.000016,0.000028,0.000063,0.000014
4,2019-01-01,sector 1,2019,13827.14,953.72,313.8455,1530.59,1125.89,400.22,725.672,...,0.000113,0.0,0.0,8.600000e-07,0.000041,0.000038,0.000016,0.000028,0.000063,0.000014


In [79]:
X=df.drop('amount_new_house_transactions',axis=1)
y=df['amount_new_house_transactions']

non_numeric = X.select_dtypes(exclude=["number"]).columns.tolist()
print("非數值欄位:", non_numeric)


非數值欄位: ['month', 'sector', 'keyword', 'source']


In [80]:
df['month'].dtypes

dtype('O')

In [81]:
df['month'] = pd.to_datetime(df['month'])
df['year'] = df['year'].astype('category')

In [82]:
X = df.drop('amount_new_house_transactions', axis=1)  # 轉型後再建
non_numeric = X.select_dtypes(exclude=['number']).columns.tolist()
print("非數值欄位:", non_numeric)

非數值欄位: ['month', 'sector', 'year', 'keyword', 'source']


In [83]:
df['month'].dtypes

dtype('<M8[ns]')

In [84]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [85]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit , cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import numpy as np

num_cols=X.select_dtypes(include=['number']).columns.tolist()
cat_cols=X.select_dtypes(exclude=['number']).columns.tolist()

pre_linear=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median')),
            ('scaler',StandardScaler())
        ]),num_cols),
    ]
)

pre_tree=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median')),
        ]),num_cols),
    ]
)


linear=LinearRegression()
pipe_linear=Pipeline([
    ('pre_linear',pre_linear),
    ('linear',linear),
])

rf=RandomForestRegressor(
    n_estimators=100,        
    max_depth=12,            
    min_samples_split=20,    
    min_samples_leaf=5,      
    max_features='sqrt',   
    bootstrap=True,          
    max_samples=0.7,         
    n_jobs=-1,
    random_state=42,
)
pipe_rf=Pipeline([
    ('pre_rf',pre_tree),
    ('rf',rf),
])

xgb=XGBRegressor(
    n_estimators=200,        
    max_depth=6,             
    learning_rate=0.1,      
    subsample=0.8,           
    colsample_bytree=0.8,    
    min_child_weight=5,      
    reg_lambda=1.0,          
    n_jobs=-1,              
    random_state=42,
    tree_method="hist"            
)
pipe_xgb=Pipeline([
    ('pre_xgb',pre_tree),
    ('xgb',xgb),
])




#設定評分方法
# —— 兩階段 MAPE：回傳「分數，越大越好」——
def two_stage_mape_score(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)

    # APE：處理 y_true=0 的情況（與常識一致：真值為 0 且預測也為 0 → 0；否則 → inf）
    ape = np.where(
        y_true == 0,
        np.where(np.isclose(y_pred, 0.0), 0.0, np.inf),
        np.abs(y_pred - y_true) / np.abs(y_true)
    )

    n = len(ape)
    frac_big = np.mean(ape > 1)  # >100% 的比例

    # 第一階段：若 >30% 的樣本 APE>1 → 直接 0 分
    if frac_big > 0.3:
        return 0.0

    # 第二階段：只用 APE<=1 的樣本，計算縮放後的 MAPE，score = 1 - scaled_mape
    mask_ok = (ape <= 1) & np.isfinite(ape)
    if mask_ok.sum() == 0:
        return 0.0  # 理論上不會到這裡，保險起見

    mape_ok = ape[mask_ok].mean()
    frac_ok = mask_ok.sum() / n
    scaled_mape = mape_ok / frac_ok
    score = 1.0 - scaled_mape

    # 分數界於 [0,1]（理論上已是如此，這裡穩健處理）
    return float(np.clip(score, 0.0, 1.0))

# 包成 sklearn scorer（越大越好）
kaggle_scorer = make_scorer(two_stage_mape_score, greater_is_better=True)


tscv=TimeSeriesSplit(n_splits=5)

cv_scores_linear=cross_val_score(
    pipe_linear,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("Linear:CV (Kaggle two-stage score) mean:", cv_scores_linear.mean())
print("Linear:Each split:", cv_scores_linear)

cv_scores_rf=cross_val_score(
    pipe_rf,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("RF:CV (Kaggle two-stage score) mean:", cv_scores_rf.mean())
print("RF:Each split:", cv_scores_rf)

cv_scores_xgb=cross_val_score(
    pipe_xgb,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("XGB:CV (Kaggle two-stage score) mean:", cv_scores_xgb.mean())
print("XGB:Each split:", cv_scores_xgb)

c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
C:\Users\USER\AppData\Local\Temp\ipykernel_24384\71088798.py:85: RuntimeWarning: divide by zero encountered in divide
  np.abs(y_pred - y_true) / np.abs(y_true)
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages

Linear:CV (Kaggle two-stage score) mean: 0.0
Linear:Each split: [0. 0. 0. 0. 0.]


c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
C:\Users\USER\AppData\Local\Temp\ipykernel_24384\71088798.py:85: RuntimeWarning: divide by zero encountered in divide
  np.abs(y_pred - y_true) / np.abs(y_true)
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages

RF:CV (Kaggle two-stage score) mean: 0.0
RF:Each split: [0. 0. 0. 0. 0.]


c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
C:\Users\USER\AppData\Local\Temp\ipykernel_24384\71088798.py:85: RuntimeWarning: divide by zero encountered in divide
  np.abs(y_pred - y_true) / np.abs(y_true)
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['total_fixed_asset_investment_10k']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages

XGB:CV (Kaggle two-stage score) mean: 0.0
XGB:Each split: [0. 0. 0. 0. 0.]


C:\Users\USER\AppData\Local\Temp\ipykernel_24384\71088798.py:85: RuntimeWarning: divide by zero encountered in divide
  np.abs(y_pred - y_true) / np.abs(y_true)


In [86]:
# 1. 全 NaN
nan_cols = X.columns[X.isna().all()]

# 2. 幾乎全 NaN（例如超過 95% 缺失）
mostly_nan_cols = X.columns[X.isna().mean() > 0.95]

# 3. 常數或接近常數欄
const_cols = X.columns[X.nunique(dropna=True) <= 1]

# 設定一個閾值，例如：非 NaN 比例 < 5% 就刪掉
low_info_cols = X.columns[(X.notna().mean() < 0.05)]

drop_cols = nan_cols.union(mostly_nan_cols).union(const_cols).union(low_info_cols)
print("要丟掉的欄位數:", len(drop_cols))

X=X.drop(columns=drop_cols)

 

要丟掉的欄位數: 13


In [87]:
print("y==0 數量:", (y == 0).sum(), "/", len(y))


y==0 數量: 65160 / 450300


In [88]:
# ===== 修正版兩階段 MAPE (處理 y_true=0) =====
def two_stage_mape_score(y_true, y_pred, epsilon=1e-6):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)

    # 避免除以零：分母加 epsilon
    ape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)

    n = len(ape)
    frac_big = np.mean(ape > 1)  # >100% 的比例

    # 第一階段：若超過 30% 的樣本 APE > 1 → 直接 0 分
    if frac_big > 0.3:
        return 0.0

    # 第二階段：只算 APE <= 1 的樣本
    mask_ok = (ape <= 1) & np.isfinite(ape)
    if mask_ok.sum() == 0:
        return 0.0

    mape_ok = ape[mask_ok].mean()
    frac_ok = mask_ok.sum() / n
    scaled_mape = mape_ok / frac_ok
    score = 1.0 - scaled_mape

    return float(np.clip(score, 0.0, 1.0))

# ===== 包成 sklearn scorer (越大越好) =====
kaggle_scorer = make_scorer(two_stage_mape_score, greater_is_better=True)


num_cols=X.select_dtypes(include=['number']).columns.tolist()
cat_cols=X.select_dtypes(exclude=['number']).columns.tolist()

pre_linear=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median')),
            ('scaler',StandardScaler())
        ]),num_cols),
    ]
)

pre_tree=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median')),
        ]),num_cols),
    ]
)


linear=LinearRegression()
pipe_linear=Pipeline([
    ('pre_linear',pre_linear),
    ('linear',linear),
])

rf=RandomForestRegressor(
    n_estimators=100,        
    max_depth=12,            
    min_samples_split=20,    
    min_samples_leaf=5,      
    max_features='sqrt',   
    bootstrap=True,          
    max_samples=0.7,         
    n_jobs=-1,
    random_state=42,
)
pipe_rf=Pipeline([
    ('pre_rf',pre_tree),
    ('rf',rf),
])

xgb=XGBRegressor(
    n_estimators=200,        
    max_depth=6,             
    learning_rate=0.1,      
    subsample=0.8,           
    colsample_bytree=0.8,    
    min_child_weight=5,      
    reg_lambda=1.0,          
    n_jobs=-1,              
    random_state=42,
    tree_method="hist"        
)
pipe_xgb=Pipeline([
    ('pre_xgb',pre_tree),
    ('xgb',xgb),
])

tscv=TimeSeriesSplit(n_splits=5)

cv_scores_linear=cross_val_score(
    pipe_linear,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("Linear:CV (Kaggle two-stage score) mean:", cv_scores_linear.mean())
print("Linear:Each split:", cv_scores_linear)

cv_scores_rf=cross_val_score(
    pipe_rf,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("RF:CV (Kaggle two-stage score) mean:", cv_scores_rf.mean())
print("RF:Each split:", cv_scores_rf)

cv_scores_xgb=cross_val_score(
    pipe_xgb,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("XGB:CV (Kaggle two-stage score) mean:", cv_scores_xgb.mean())
print("XGB:Each split:", cv_scores_xgb)

Linear:CV (Kaggle two-stage score) mean: 0.0
Linear:Each split: [0. 0. 0. 0. 0.]
RF:CV (Kaggle two-stage score) mean: 0.0
RF:Each split: [0. 0. 0. 0. 0.]
XGB:CV (Kaggle two-stage score) mean: 0.0
XGB:Each split: [0. 0. 0. 0. 0.]


In [89]:
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
# 假設：X(含 month, sector, y_col)、features、pipeline 都已經準備好

def frac_big_ape(y_true, y_pred, eps=1e-6):
    ape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), eps)
    return float((ape > 1).mean())

tscv = TimeSeriesSplit(n_splits=5)
ratios = []
for tr_idx, te_idx in tscv.split(X):
    Xtr, Xte = X.iloc[tr_idx], X.iloc[te_idx]
    ytr, yte = y.iloc[tr_idx], y.iloc[te_idx]
    mdl = pipe_linear.fit(Xtr[num_cols], ytr)
    pred = mdl.predict(Xte[num_cols])
    pred = np.clip(pred, 0.0, None)  # y≥0
    ratios.append(frac_big_ape(yte.values, pred))

print("各折 >100% 比例：", ratios, "平均：", np.mean(ratios))


各折 >100% 比例： [0.22371752165223185, 0.34900732844770155, 0.37534976682211857, 0.31491005996002663, 0.3866222518321119] 平均： 0.32992138574283814


In [90]:
def frac_big_ape(y_true, y_pred, eps=1e-6):
    ape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), eps)
    return float((ape > 1).mean())

tscv = TimeSeriesSplit(n_splits=5)
ratios = []
for tr_idx, te_idx in tscv.split(X):
    Xtr, Xte = X.iloc[tr_idx], X.iloc[te_idx]
    ytr, yte = y.iloc[tr_idx], y.iloc[te_idx]
    mdl = pipe_rf.fit(Xtr[num_cols], ytr)
    pred = mdl.predict(Xte[num_cols])
    pred = np.clip(pred, 0.0, None)  # y≥0
    ratios.append(frac_big_ape(yte.values, pred))

print("各折 >100% 比例：", ratios, "平均：", np.mean(ratios))

各折 >100% 比例： [0.3487008660892738, 0.40924716855429716, 0.33557628247834775, 0.45169886742171883, 0.4340972684876749] 平均： 0.3958640906062625


In [91]:
def frac_big_ape(y_true, y_pred, eps=1e-6):
    ape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), eps)
    return float((ape > 1).mean())

tscv = TimeSeriesSplit(n_splits=5)
ratios = []
for tr_idx, te_idx in tscv.split(X):
    Xtr, Xte = X.iloc[tr_idx], X.iloc[te_idx]
    ytr, yte = y.iloc[tr_idx], y.iloc[te_idx]
    mdl = pipe_xgb.fit(Xtr[num_cols], ytr)
    pred = mdl.predict(Xte[num_cols])
    pred = np.clip(pred, 0.0, None)  # y≥0
    ratios.append(frac_big_ape(yte.values, pred))

print("各折 >100% 比例：", ratios, "平均：", np.mean(ratios))

各折 >100% 比例： [0.4108061292471685, 0.34739506995336444, 0.3012658227848101, 0.39949367088607596, 0.3853431045969354] 平均： 0.36886075949367086


In [92]:
from sklearn.dummy import DummyRegressor

dummy_reg=DummyRegressor(strategy='median')
pipe_dummy_reg=Pipeline([
    ('pre_dummy_reg',pre_tree),
    ('dummy_reg',dummy_reg),
])
cv_scores_dummy_ref=cross_val_score(
    pipe_dummy_reg,X[num_cols],y,cv=tscv,scoring=kaggle_scorer
)
print("Dummy:CV (Kaggle two-stage score) mean:", cv_scores_dummy_ref.mean())
print("Dummy:Each split:", cv_scores_dummy_ref)

Dummy:CV (Kaggle two-stage score) mean: 0.0
Dummy:Each split: [0. 0. 0. 0. 0.]


In [93]:
df["year_month"] = pd.to_datetime(df["month"]).dt.to_period("M").dt.to_timestamp()
df = df.sort_values(["sector", "year_month"]).reset_index(drop=True)

X = df.drop("amount_new_house_transactions", axis=1)
y = df["amount_new_house_transactions"]



In [94]:
date_col  = "year_month"   
group_col = "sector"

g = df.groupby(group_col)["amount_new_house_transactions"]


#建立天真基線
preds = {
    "lag1":  g.shift(1),
    "lag12": g.shift(12),
    "ma3":   g.shift(1).rolling(3).mean(),
    "ma6":   g.shift(1).rolling(6).mean(),
}

#評估
rows = []
y_true = y.to_numpy(float)
for name, p in preds.items():
    y_pred = np.where(np.isnan(p), 0.0, p)  # 前幾期沒有歷史 → 補0
    y_pred = np.clip(y_pred, 0.0, None)     # y≥0
    score  = two_stage_mape_score(y_true, y_pred)
    ratio  = frac_big_ape(y_true, y_pred)
    rows.append((name, score, ratio))

print(pd.DataFrame(rows, columns=["baseline","two_stage_score","frac_APE>100%"])
        .sort_values("two_stage_score", ascending=False))

  baseline  two_stage_score  frac_APE>100%
0     lag1         0.995663       0.002385
2      ma3         0.989868       0.005241
3      ma6         0.981318       0.009460
1    lag12         0.945110       0.028621


In [95]:
#刪除NaN值的X欄
# 1. 全 NaN
nan_cols = df.columns[df.isna().all()]

# 2. 幾乎全 NaN（例如超過 95% 缺失）
mostly_nan_cols = df.columns[df.isna().mean() > 0.95]

# 3. 常數或接近常數欄
const_cols = df.columns[df.nunique(dropna=True) <= 1]

# 設定一個閾值，例如：非 NaN 比例 < 5% 就刪掉
low_info_cols = df.columns[(df.notna().mean() < 0.05)]

drop_cols = nan_cols.union(mostly_nan_cols).union(const_cols).union(low_info_cols)

df=df.drop(columns=drop_cols)





In [96]:
# === 安裝工具包 ===
from lag_rolling_tools import GroupTimeLagRoller

# === 定義欄位 ===
date_col  = "year_month"
group_col = "sector"
y_col     = ["amount_new_house_transactions"]

cat_cols = ['month', 'sector', 'year', 'keyword', 'source']

# 數值欄位: 自動選 numeric & 排除掉 y, group, date
exclude = set(cat_cols + y_col + [date_col, group_col])
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in exclude]

# === Lag 特徵生成器 ===
lagrolls = GroupTimeLagRoller(
    group_key=group_col,
    time_col=date_col,
    cols=y_col,
    lags=[1, 3, 12],
    roll_windows=[3, 6],
    roll_funcs=['mean'],
    shift_for_rolling=1,
    drop_original_y=True,
)

lag_cols = lagrolls.get_column_names()

# === Column Selector: 排除 lag 用來 impute ===
def select_num_excl_lag(X):
    numeric = X.select_dtypes(include=[np.number]).columns
    return [c for c in numeric if c not in set(lag_cols)]

# === ColumnTransformer ===
pre_linear=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median')),
            ('scaler',StandardScaler())
        ]),select_num_excl_lag),
        ('lag',SimpleImputer(strategy='constant',fill_value=0),lag_cols),
    ],
    remainder='drop'
)

pre_tree = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median'))
        ]), select_num_excl_lag),
        ('lag',SimpleImputer(strategy='constant',fill_value=0),lag_cols),
    ],
    remainder='drop'
)

# === Pipeline ===
pipe_linear_lag=Pipeline([
    ('lagrolls',lagrolls),
    ('pre',pre_linear),
    ('linear',linear)
])

pipe_rf_lag=Pipeline([
    ('lagrolls',lagrolls),
    ('pre',pre_tree),
    ('rf',rf)
])

pipe_xgb_lag = Pipeline([
    ('lagrolls', lagrolls),
    ('pre', pre_tree),
    ('xgb', xgb)   
])

# === Cross Validation ===
X = df
y = df[y_col[0]]

tscv = TimeSeriesSplit(n_splits=5)

cv_scores_linear=cross_val_score(pipe_linear_lag,X=X,y=y,cv=tscv,scoring=kaggle_scorer)
print("linear CV mean:", cv_scores_linear.mean())
print("linear CV each split:", cv_scores_linear)

cv_scores_rf=cross_val_score(pipe_rf_lag,X=X,y=y,cv=tscv,scoring=kaggle_scorer)
print("RF CV mean:", cv_scores_rf.mean())
print("RF CV each split:", cv_scores_rf)

cv_scores_xgb = cross_val_score(pipe_xgb_lag, X=X, y=y, cv=tscv, scoring=kaggle_scorer)

print("XGB CV mean:", cv_scores_xgb.mean())
print("XGB CV each split:", cv_scores_xgb)


linear CV mean: 0.8909493073294396
linear CV each split: [0.95472145 0.91389829 0.8880529  0.82902952 0.86904438]
RF CV mean: 0.2934185613385213
RF CV each split: [0.72686491 0.         0.74022789 0.         0.        ]
XGB CV mean: 0.8964605311574093
XGB CV each split: [0.86925971 0.86879327 0.93166086 0.89678574 0.91580307]


In [97]:
%pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [98]:
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn import FunctionSampler
# === 定義欄位 ===
date_col  = "year_month"
group_col = "sector"
y_col     = ["amount_new_house_transactions"]

cat_cols = ['month', 'sector', 'year', 'keyword', 'source']

# 數值欄位: 自動選 numeric & 排除掉 y, group, date
exclude = set(cat_cols + y_col + [date_col, group_col])
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in exclude]

# === Lag 特徵生成器 ===
lagrolls = GroupTimeLagRoller(
    group_key=group_col,
    time_col=date_col,
    cols=y_col,
    lags=[1, 3, 12],
    roll_windows=[3, 6],
    roll_funcs=['mean'],
    shift_for_rolling=1,
    drop_original_y=True,
)

lag_cols = lagrolls.get_column_names()

# 2) 刪暖機段：在 lag 後、pre_tree 前，同步處理 X 與 y
def drop_warmup_rows(X, y, lag_columns):
    # X 此時是 DataFrame（因為前一步是你自訂的 transformer）
    mask = X[lag_columns].notna().all(axis=1)
    # 同步篩 X 與 y（保留 index 對齊）
    return X.loc[mask], y.loc[mask]

dropper = FunctionSampler(
    func=drop_warmup_rows,
    kw_args={'lag_columns': lag_cols},
    validate=False   # ← 關鍵：不要把 y 當分類標籤檢查
)

# === Column Selector: 排除 lag 用來 impute ===
def select_num_excl_lag(X):
    numeric = X.select_dtypes(include=[np.number]).columns
    return [c for c in numeric if c not in set(lag_cols)]

# === ColumnTransformer ===
pre_linear=ColumnTransformer(
    transformers=[
        ('num',Pipeline([
            ('imputer',SimpleImputer(strategy='median')),
            ('scaler',StandardScaler())
        ]),select_num_excl_lag),
        ('lag', SimpleImputer(strategy='constant',fill_value=0), lag_cols),
    ],
    remainder='drop'
)


pre_tree = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median'))
        ]), select_num_excl_lag),
        ('lag', 'passthrough', lag_cols),
    ],
    remainder='drop'
)


# === Pipeline ===
pipe_linear_lag=ImbPipeline([
    ('lagrolls',lagrolls),
    ('dropper',dropper),
    ('pre',pre_linear),
    ('linear',linear)
])

pipe_rf_lag=ImbPipeline([
    ('lagrolls',lagrolls),
    ('dropper',dropper),
    ('pre',pre_tree),
    ('rf',rf)
])

pipe_xgb_lag = ImbPipeline([
    ('lagrolls', lagrolls),
    ('dropper',dropper),
    ('pre', pre_tree),
    ('xgb', xgb)   # 這裡放你的 XGBRegressor
])

# === Cross Validation ===
X = df
y = df[y_col[0]]

tscv = TimeSeriesSplit(n_splits=5)
cv_scores_linear=cross_val_score(pipe_linear_lag,X=X,y=y,cv=tscv,scoring=kaggle_scorer)
print("linear CV mean:", cv_scores_linear.mean())
print("linear CV each split:", cv_scores_linear)

cv_scores_rf=cross_val_score(pipe_rf_lag,X=X,y=y,cv=tscv,scoring=kaggle_scorer)
print("RF CV mean:", cv_scores_rf.mean())
print("RF CV each split:", cv_scores_rf)

cv_scores_xgb = cross_val_score(pipe_xgb_lag, X=X, y=y, cv=tscv, scoring=kaggle_scorer)

print("XGB CV mean:", cv_scores_xgb.mean())
print("XGB CV each split:", cv_scores_xgb)

linear CV mean: 0.8909808407714385
linear CV each split: [0.9540819  0.91451592 0.89341048 0.82454542 0.8683505 ]
RF CV mean: 0.30201793995685555
RF CV each split: [0.74679763 0.         0.76329207 0.         0.        ]
XGB CV mean: 0.8864154058832513
XGB CV each split: [0.87536771 0.81959113 0.92981984 0.89693062 0.91036774]


In [99]:
# 確認XGB，NaN補0為baseline


# === 定義欄位 ===
date_col  = "year_month"
group_col = "sector"
y_col     = ["amount_new_house_transactions"]

cat_cols = ['month', 'sector', 'year', 'keyword', 'source']

# 數值欄位: 自動選 numeric & 排除掉 y, group, date
exclude = set(cat_cols + y_col + [date_col, group_col])
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in exclude]

# === Lag 特徵生成器 ===
lagrolls = GroupTimeLagRoller(
    group_key=group_col,
    time_col=date_col,
    cols=y_col,
    lags=[1, 3, 12],
    roll_windows=[3, 6],
    roll_funcs=['mean'],
    shift_for_rolling=1,
    drop_original_y=True,
)

lag_cols = lagrolls.get_column_names()

# === Column Selector: 排除 lag 用來 impute ===
def select_num_excl_lag(X):
    numeric = X.select_dtypes(include=[np.number]).columns
    return [c for c in numeric if c not in set(lag_cols)]

# === ColumnTransformer ===
pre_tree = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median'))
        ]), select_num_excl_lag),
        ('lag',SimpleImputer(strategy='constant',fill_value=0),lag_cols),
    ],
    remainder='drop'
)

# === Pipeline ===
pipe_xgb_lag = Pipeline([
    ('lagrolls', lagrolls),
    ('pre', pre_tree),
    ('xgb', xgb)   
])

# === Cross Validation ===
X = df
y = df[y_col[0]]

tscv = TimeSeriesSplit(n_splits=5)

cv_scores_xgb = cross_val_score(pipe_xgb_lag, X=X, y=y, cv=tscv, scoring=kaggle_scorer)

print("XGB CV mean:", cv_scores_xgb.mean())
print("XGB CV each split:", cv_scores_xgb)

XGB CV mean: 0.8964605311574093
XGB CV each split: [0.86925971 0.86879327 0.93166086 0.89678574 0.91580307]


In [100]:
# === 1) 先把管線 fit 一次（可用整體或你切好的 train）===
pipe_xgb_lag.fit(X, y)

# === 2) 取出「前處理」後的特徵名稱 ===
# 先經過 lag/rolling 轉換，再問 ColumnTransformer 要欄名
X_after_lag = pipe_xgb_lag.named_steps['lagrolls'].transform(X)

pre = pipe_xgb_lag.named_steps['pre']
# sklearn >= 1.0 支援 get_feature_names_out；若你的版本較舊，見下面備案
feature_names = pre.get_feature_names_out(input_features=getattr(X_after_lag, "columns", None))

# === 3) 從 XGB 取得重要度並對上欄名 ===
import numpy as np
import pandas as pd

xgb_model = pipe_xgb_lag.named_steps['xgb']
fi = pd.Series(xgb_model.feature_importances_, index=feature_names).sort_values(ascending=False)

# 看前 30 名
top_n = 30
print(fi.head(top_n))


lag__amount_new_house_transactions_lag1                          0.584319
lag__amount_new_house_transactions_rollmean3_s1_mp3              0.075728
lag__amount_new_house_transactions_rollmean6_s1_mp6              0.074697
num__education_training_school_education_research_institution    0.030558
num__surrounding_housing_average_price                           0.027720
num__num_pre_owned_house_transactions_nearby_sectors             0.017504
num__price_pre_owned_house_transactions                          0.016917
num__surrounding_shop_average_rent                               0.016227
num__area_pre_owned_house_transactions_nearby_sectors            0.015392
num__price_new_house_transactions_nearby_sectors                 0.010232
num__price_pre_owned_house_transactions_nearby_sectors           0.009477
num__office_population                                           0.008139
num__period_new_house_sell_through_nearby_sectors                0.007307
num__area_per_unit_new_house_transacti

In [ ]:
# 嘗試只保留lag/rolling特徵


# === 定義欄位 ===
date_col  = "year_month"
group_col = "sector"
y_col     = ["amount_new_house_transactions"]

cat_cols = ['month', 'sector', 'year', 'keyword', 'source']

# 數值欄位: 自動選 numeric & 排除掉 y, group, date
exclude = set(cat_cols + y_col + [date_col, group_col])
num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in exclude]

# === Lag 特徵生成器 ===
lagrolls = GroupTimeLagRoller(
    group_key=group_col,
    time_col=date_col,
    cols=y_col,
    lags=[1, 3, 12],
    roll_windows=[3, 6],
    roll_funcs=['mean'],
    shift_for_rolling=1,
    drop_original_y=True,
    return_only_new_features=True
)

lag_cols = lagrolls.get_column_names()

# === Column Selector: 排除 lag 用來 impute ===
def select_num_excl_lag(X):
    numeric = X.select_dtypes(include=[np.number]).columns
    return [c for c in numeric if c not in set(lag_cols)]

# === ColumnTransformer ===
pre_tree = ColumnTransformer(
    transformers=[
        ('lag',SimpleImputer(strategy='constant',fill_value=0),lag_cols),
    ],
    remainder='drop'
)

# === Pipeline ===
pipe_xgb_lag = Pipeline([
    ('lagrolls', lagrolls),
    ('pre', pre_tree),
    ('xgb', xgb)   
])

# === Cross Validation ===
X = df
y = df[y_col[0]]

tscv = TimeSeriesSplit(n_splits=5)

cv_scores_xgb = cross_val_score(pipe_xgb_lag, X=X, y=y, cv=tscv, scoring=kaggle_scorer)

print("XGB CV mean:", cv_scores_xgb.mean())
print("XGB CV each split:", cv_scores_xgb)

XGB CV mean: 0.9655650285767526
XGB CV each split: [0.96885715 0.96439589 0.96714527 0.95672104 0.97070579]


In [104]:
from scipy.stats import randint, uniform, loguniform
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd

# --- 1) 超參數搜尋空間（XGBRegressor in Pipeline -> 'xgb__*'） ---
param_dist = {
    "xgb__n_estimators": randint(300, 1200),
    "xgb__max_depth": randint(3, 10),
    "xgb__learning_rate": loguniform(1e-3, 3e-1),
    "xgb__subsample": uniform(0.6, 0.4),          # 0.6 ~ 1.0
    "xgb__colsample_bytree": uniform(0.6, 0.4),   # 0.6 ~ 1.0
    "xgb__min_child_weight": loguniform(1e-1, 1e2),
    "xgb__reg_alpha": loguniform(1e-4, 1e1),
    "xgb__reg_lambda": loguniform(1e-3, 1e1),
    "xgb__gamma": loguniform(1e-8, 1e-1),
}

# --- 2) 建立 RandomizedSearchCV ---
rs = RandomizedSearchCV(
    estimator=pipe_xgb_lag,
    param_distributions=param_dist,
    n_iter=30,                
    cv=tscv,                  
    scoring=kaggle_scorer,    
    n_jobs=-1,
    verbose=1,
    random_state=42,
    refit=True,                # 以最佳參數在全資料上重訓
    error_score="raise"        # 有錯直接丟出來好除錯
)

# --- 3) 執行搜尋 ---
rs.fit(X, y)

print("Best score (CV):", rs.best_score_)
print("Best params:")
for k, v in rs.best_params_.items():
    print(f"  {k}: {v}")

# 最佳模型（已用最佳參數 refit 完成）
best_model = rs.best_estimator_


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best score (CV): 0.9688623308934974
Best params:
  xgb__colsample_bytree: 0.6147547789418131
  xgb__gamma: 0.0001849057164788623
  xgb__learning_rate: 0.01758720770925141
  xgb__max_depth: 8
  xgb__min_child_weight: 0.1256470217114413
  xgb__n_estimators: 1035
  xgb__reg_alpha: 3.477998816092334
  xgb__reg_lambda: 0.009083381663660588
  xgb__subsample: 0.6579579488364892


In [105]:
# 1) 取最佳參數，建立最終管線並重訓
best_params = rs.best_params_
pipe_final = pipe_xgb_lag.set_params(**best_params)

# 全資料重訓（與 rs.refit 類似，但我們明確再跑一次）
pipe_final.fit(X, y)

# 2) 存檔（models/ 目錄）
import joblib, os
os.makedirs("models", exist_ok=True)
joblib.dump(pipe_final, "models/xgb_lag_pipeline.joblib", compress=3)

print("Saved -> models/xgb_lag_pipeline.joblib")


Saved -> models/xgb_lag_pipeline.joblib


In [106]:
import sklearn, xgboost, pandas, numpy
print("sklearn", sklearn.__version__)
print("xgboost", xgboost.__version__)
print("pandas", pandas.__version__)
print("numpy", numpy.__version__)


sklearn 1.6.1
xgboost 3.0.4
pandas 2.2.3
numpy 2.1.3
